In [1]:
import numpy as np
import tensorflow as tf

## TensorFlow Core 演示
您可以将 TensorFlow Core 程序看作由两个互相独立的部分组成：

1. 构建计算图 (`tf.Graph`)。
2. 运行计算图（`使用 tf.Session`）。
### 图
**计算图**是排列成一个图的一系列 TensorFlow 指令。图由两种类型的对象组成。

- 操作（简称“op”）：图的节点。操作描述了消耗和生成张量的计算。
- 张量：图的边。它们代表将流经图的值。大多数 TensorFlow 函数会返回 `tf.Tensors`。

> 重要提示：tf.Tensors 不具有值，它们只是计算图中元素的手柄。

我们来构建一个简单的计算图。最基本的指令是一个常量。构建指令的 Python 函数将一个张量值作为输入值。生成的指令不需要输入值。它在运行时输出的是被传递给构造函数的值。我们可以创建如下所示的两个浮点数常量 `a` 和 `b`：

In [2]:
a = tf.constant(3.0,dtype=tf.float32)
b = tf.constant(4.0) # also tf.tfloat32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


请注意，打印张量并不会如您可能预期的那样输出值 3.0、4.0 和 7.0。上述语句只会构建计算图。这些 `tf.Tensor` 对象仅代表将要运行的操作的结果。

图中的每个指令都拥有唯一的名称。这个名称不同于使用 Python 分配给相应对象的名称。张量是根据生成它们的指令命名的，后面跟着输出索引，如上文的 "add:0" 所示。

##### TensorBoard
TensorFlow 提供了一个名为 TensorBoard 的实用程序。TensorBoard 的诸多功能之一是将计算图可视化。您只需要使用几个简单的命令就能轻松完成此操作。

首先将计算图保存为 TensorBoard 摘要文件，具体操作如下所示：

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

这将在当前目录中生成一个 event 文件，其名称格式如下：

> events.out.tfevents.{timestamp}.{hostname}

现在，在新的终端中使用以下 shell 命令启动 TensorBoard：

> tensorboard --logdir .

接下来，在您的浏览器中打开 TensorBoard 的[图页面](http://localhost:6006/#graphs)，您应该会看到与以下图形类似的图：

![TensorBoard screenshot](https://tensorflow.google.cn/images/getting_started_add.png)

要详细了解 TensorBoard 的计算图可视化工具，请参阅 [TensorBoard：图的直观展示](https://tensorflow.google.cn/guide/graph_viz)。

### 会话 (Session)
要评估张量，需要实例化一个 `tf.Session` 对象（非正式名称为会话）。会话会封装 TensorFlow 运行时的状态，并运行 TensorFlow 操作。如果说 `tf.Graph` 像一个 `.py` 文件，那么` tf.Session` 就像一个 python 可执行对象。

下面的代码会创建一个 `tf.Session` 对象，然后调用其 `run` 方法来评估我们在上文中创建的 `total` 张量：

In [5]:
sess = tf.Session()
print(sess.run(total))

7.0


当您使用 `Session.run` 请求输出节点时，TensorFlow 会回溯整个图，并流经提供了所请求的输出节点对应的输入值的所有节点。因此此指令会打印预期的值 7.0：

---
您可以将多个张量传递给 `tf.Session.run`。run 方法以透明方式处理元组或字典的任何组合，如下例所示：

In [6]:
print(sess.run({'ab':(a,b),'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


在调用 `tf.Session.run` 期间，任何 `tf.Tensor` 都只有单个值。例如，以下代码调用 `tf.random_uniform` 来生成一个 `tf.Tensor`，后者会生成随机的三元素矢量（值位于 `[0,1)` 区间内）：

vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1,out2)))

部分 TensorFlow 函数会返回 `tf.Operations`，而不是 `tf.Tensors`。对指令调用 `run` 的结果是 None。您运行指令是为了产生副作用，而不是为了检索一个值。这方面的例子包括稍后将演示的初始化和训练操作。

### 供给
目前来讲，这个图不是特别有趣，因为它总是生成一个常量结果。图可以参数化以便接受外部输入，也称为**占位符**。**占位符**表示承诺在稍后提供值，它就像函数参数。

In [11]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

前面三行有点像函数。我们定义了这个函数的两个输入参数（`x` 和 `y`），然后对它们运行指令。我们可以使用 `run` 方法的 `feed_dict` 参数为占位符提供具体的值，从而评估这个具有多个输入的图：

In [12]:
print(sess.run(z,feed_dict={x:3,y:4.5}))
print(sess.run(z,feed_dict={x:[1,3],y:[2,4]}))

7.5
[3. 7.]


另请注意，`feed_dict `参数可用于覆盖图中的任何张量。占位符和其他 `tf.Tensors` 的唯一不同之处在于如果没有为占位符提供值，那么占位符会抛出错误。

##### 数据集
占位符适用于简单的实验，而数据集是将数据流式传输到模型的首选方法。

要从数据集中获取可运行的 `tf.Tensor`，您必须先将其转换成 `tf.data.Iterator`，然后调用迭代器的 `get_next` 方法。

创建迭代器的最简单的方式是采用 `make_one_shot_iterator` 方法。例如，在下面的代码中，`next_item` 张量将在每次` run `调用时从 `my_data` 阵列返回一行：

In [13]:
my_data = [
    [0,1,],
    [2,3,],
    [4,5,],
    [6,7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

到达数据流末端时，`Dataset` 会抛出 `OutOfRangeError`。例如，下面的代码会一直读取 `next_item`，直到没有数据可读：

In [14]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break
            

[0 1]
[2 3]
[4 5]
[6 7]


如果 `Dataset` 依赖于有状态操作，则可能需要在使用迭代器之前先初始化它，如下所示：

In [15]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

Instructions for updating:
Colocations handled automatically by placer.
[-0.01796639  0.5418985   0.5921779 ]
[-0.26439765 -1.0812902   0.15834706]
[ 0.686885   -0.03923053  0.32282972]
[ 1.9284716  -0.6536119   0.08343184]
[-0.95533794  0.16460852  1.7676501 ]
[ 0.8603501 -2.3254235 -0.9012111]
[ 0.65457284  0.41056755 -0.2854925 ]
[ 1.235059   -0.15938279 -0.2751244 ]
[-0.6736671   0.16163778 -0.5215337 ]
[-1.8971568  1.323353   1.438493 ]


### 层
可训练的模型必须修改图中的值，以便在输入相同值的情况下获得新的输出值。将可训练参数添加到图中的首选方法是**层**。

层将变量和作用于它们的操作打包在一起。例如，**密集连接层**会对每个输出对应的所有输入执行加权和，并应用**激活函数**（可选）。连接权重和偏差由层对象管理。

### 创建层
下面的代码会创建一个 `Dense` 层，该层会接受一批输入矢量，并为每个矢量生成一个输出值。要将层应用于输入值，请将该层当做函数来调用。例如：

In [16]:
x = tf.placeholder(tf.float32,shape=[None,3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

层会检查其输入数据，以确定其内部变量的大小。因此，我们必须在这里设置` x` 占位符的形状，以便层构建正确大小的权重矩阵。

我们现在已经定义了输出值 `y `的计算，在我们运行计算之前，还需要处理一个细节。

### 初始化层
层包含的变量必须先**初始化**，然后才能使用。尽管可以单独初始化各个变量，但也可以轻松地初始化一个 TensorFlow 图中的所有变量（如下所示）：

In [17]:
init = tf.global_variables_initializer()
sess.run(init)

>重要提示：调用 `tf.global_variables_initializer` 仅会创建并返回 TensorFlow 操作的句柄。当我们使用 tf.Session.run 运行该操作时，该操作将初始化所有全局变量。

另请注意，此 `global_variables_initializer` 仅会初始化创建初始化程序时图中就存在的变量。因此您应该在构建图表的最后一步添加初始化程序。

### 执行层
我们现在已经完成了层的初始化，可以像处理任何其他张量一样评估 `linear_model` 的输出张量了。例如，下面的代码：

In [18]:
print(sess.run(y,{x:[[1,2,3],[4,5,6]]}))

[[ 4.0764174]
 [10.157795 ]]


层函数的快捷方式
对于每个层类（如 tf.layers.Dense)，TensorFlow 还提供了一个快捷函数（如 tf.layers.dense）。两者唯一的区别是快捷函数版本是在单次调用中创建和运行层。例如，以下代码等同于较早的版本：